## Configuração e imports


In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.feature_selection import SelectKBest, f_classif, RFE, SelectFromModel
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

## Carregamento de dados e definição de X e y


In [2]:
# Carrega os dados
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print(f"Shape treino: {train.shape}, Shape teste: {test.shape}")
print(f"Distribuição do target: {train['labels'].value_counts().to_dict()}")
print(f"Proporção classe 1: {train['labels'].mean():.3f}")

X = train.drop('labels', axis=1)
y = train['labels']


Shape treino: (646, 33), Shape teste: (277, 32)
Distribuição do target: {1: 418, 0: 228}
Proporção classe 1: 0.647


## Engenharia de atributos


In [3]:
# Feature Engineering Avançado
def create_features(df):
    df = df.copy()

    # Features de tempo
    df['funding_duration'] = df['age_last_funding_year'] - df['age_first_funding_year']
    df['milestone_duration'] = df['age_last_milestone_year'] - df['age_first_milestone_year']
    df['time_to_first_milestone'] = df['age_first_milestone_year'] - df['age_first_funding_year']

    # Features de funding por round
    df['funding_per_round'] = df['funding_total_usd'] / (df['funding_rounds'] + 1e-6)
    df['milestones_per_round'] = df['milestones'] / (df['funding_rounds'] + 1e-6)

    # Features de relacionamentos
    df['relationships_per_round'] = df['relationships'] / (df['funding_rounds'] + 1e-6)
    df['participants_per_round'] = df['avg_participants'] * df['funding_rounds']

    # Features de funding por milestone
    df['funding_per_milestone'] = df['funding_total_usd'] / (df['milestones'] + 1e-6)

    # Features de estado (one-hot já existe, mas vamos criar features combinadas)
    df['is_major_state'] = df['is_CA'] + df['is_NY'] + df['is_MA'] + df['is_TX']

    # Features de categoria (one-hot já existe, mas vamos criar features combinadas)
    df['is_tech_category'] = df['is_software'] + df['is_web'] + df['is_mobile']
    df['is_business_category'] = df['is_enterprise'] + df['is_advertising'] + df['is_consulting']
    df['is_consumer_category'] = df['is_gamesvideo'] + df['is_ecommerce']

    # Features de funding rounds
    df['has_multiple_rounds'] = (df['funding_rounds'] > 1).astype(int)
    df['has_advanced_rounds'] = (df['has_roundC'] + df['has_roundD']).astype(int)

    # Features de milestones
    df['has_milestones'] = (df['milestones'] > 0).astype(int)
    df['milestone_intensity'] = df['milestones'] / (df['age_last_funding_year'] - df['age_first_funding_year'] + 1e-6)

    # Features de funding total (log scale)
    df['log_funding_total'] = np.log1p(df['funding_total_usd'])

    # Features de idade (log scale)
    df['log_age_first_funding'] = np.log1p(df['age_first_funding_year'])
    df['log_age_last_funding'] = np.log1p(df['age_last_funding_year'])

    return df


In [4]:
print("\nFEATURE ENGINEERING")
X_engineered = create_features(X)
test_engineered = create_features(test)

print(f"Features originais: {X.shape[1]}")
print(f"Features após engenharia: {X_engineered.shape[1]}")

# Seleciona colunas numéricas e categóricas
numeric_features = X_engineered.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_engineered.select_dtypes(include=['object']).columns.tolist()

print(f"Features numéricas: {len(numeric_features)}")
print(f"Features categóricas: {len(categorical_features)}")



FEATURE ENGINEERING
Features originais: 32
Features após engenharia: 51
Features numéricas: 50
Features categóricas: 1


## Pré-processamento


In [5]:
# Pré-processamento mais sofisticado
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

print("Pipeline de pré-processamento configurado:")
print(f"- Features numéricas: {len(numeric_features)}")
print(f"- Features categóricas: {len(categorical_features)}")
print("- Imputação: KNN para numéricas, moda para categóricas")
print("- Escalonamento: RobustScaler para numéricas")

Pipeline de pré-processamento configurado:
- Features numéricas: 50
- Features categóricas: 1
- Imputação: KNN para numéricas, moda para categóricas
- Escalonamento: RobustScaler para numéricas


## Definição de modelos


In [6]:
# Modelos individuais ajustados para maximizar a acurácia

# Random Forest padrão
rf = RandomForestClassifier(
    n_estimators=300,           # Mais árvores para melhor generalização
    max_depth=15,               # Controla overfitting
    min_samples_split=5,        # Evita divisões muito específicas
    min_samples_leaf=2,         # Garante folhas com amostras suficientes
    max_features='sqrt',        # Usa sqrt(n_features) para cada split
    bootstrap=True,             # Bootstrap para diversidade
    class_weight='balanced',    # Lida com desbalanceamento (64.7% classe 1)
    random_state=42,
    n_jobs=-1
)

# Gradient Boosting com hiperparâmetros ajustados
gb = GradientBoostingClassifier(
    n_estimators=2500,      # Mais árvores
    learning_rate=0.02,     # Menor taxa de aprendizado para maior precisão
    max_depth=12,           # Aumenta profundidade
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.8,          # Mais amostras por árvore
    random_state=42
)

# Extra Trees padrão
et = ExtraTreesClassifier(
    n_estimators=300,           # Mais árvores para melhor generalização
    max_depth=20,               # Extra Trees pode ter mais profundidade (mais aleatório)
    min_samples_split=2,        # Menor que RF (Extra Trees é mais tolerante)
    min_samples_leaf=1,         # Menor que RF (Extra Trees é mais tolerante)
    max_features='sqrt',        # Usa sqrt(n_features) para cada split
    bootstrap=True,             # Bootstrap para diversidade
    class_weight='balanced',    # Lida com desbalanceamento
    random_state=42,
    n_jobs=-1
)

# Ajustando os hiperparâmetros do LogisticRegression para maximizar a acurácia
lr = LogisticRegression(
    C=1.5,  # Aumentando C para menos regularização, geralmente melhora a acurácia
    solver='lbfgs',  # Solver eficiente para conjuntos de dados não esparsos
    penalty='l2',  # Regularização padrão
    class_weight='balanced',
    max_iter=3000,  # Mais iterações para garantir convergência
    random_state=42,
    n_jobs=-1
)

# SVM ajustado para acurácia máxima
svm = SVC(
    C=2.0,                  # Menos regularização
    kernel='rbf',
    gamma='scale',          # Gamma padrão
    class_weight='balanced',
    probability=True,
    random_state=42
)

# KNN ajustado para acurácia máxima
knn = KNeighborsClassifier(
    n_neighbors=5,
    weights='distance',
    metric='minkowski',
    p=2                     # Distância Euclidiana
)

# Ensemble Voting Classifier ajustado
voting_clf = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('et', et),
        ('lr', lr),
        ('svm', svm),
        ('knn', knn)
    ],
    voting='soft',
    weights=[2,2,2,1,1,1]   # Dá mais peso para ensembles de árvore
)

# Bagging com o melhor modelo individual (Random Forest)
bagging_clf = BaggingClassifier(
    estimator=rf,
    n_estimators=20,        # Mais estimadores para maior estabilidade
    random_state=42,
    n_jobs=-1
)

models = {
    'rf': rf,
    'gb': gb,
    'et': et,
    'lr': lr,
    'svm': svm,
    'knn': knn,
    'voting': voting_clf,
    'bagging': bagging_clf
}

print("Modelos configurados para máxima acurácia:")
for name, model in models.items():
    print(f"- {name}: {type(model).__name__}")


Modelos configurados para máxima acurácia:
- rf: RandomForestClassifier
- gb: GradientBoostingClassifier
- et: ExtraTreesClassifier
- lr: LogisticRegression
- svm: SVC
- knn: KNeighborsClassifier
- voting: VotingClassifier
- bagging: BaggingClassifier


## Validação cruzada e seleção do melhor modelo


In [7]:
print("\nVALIDAÇÃO CRUZADA (score: accuracy)")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
best_model = None
best_score = 0

for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    scores = cross_val_score(pipeline, X_engineered, y, cv=cv, scoring='accuracy')
    mean_score = scores.mean()
    std_score = scores.std()

    print(f"{name}: {mean_score:.4f} (+/- {std_score:.4f})")

    if mean_score > best_score:
        best_score = mean_score
        best_model = name

print(f"\nMelhor modelo: {best_model} com score: {best_score:.4f}")

# Treina o melhor modelo
best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', models[best_model])
])

best_pipeline.fit(X_engineered, y)


VALIDAÇÃO CRUZADA (score: accuracy)
rf: 0.7709 (+/- 0.0226)


KeyboardInterrupt: 

## Sistemas de votação (ensembles)


### Ensemble com todos os modelos individuais e melhores modelos


In [ ]:
# 1. Ensemble com todos os modelos individuais
all_individual_models = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('et', et),
        ('lr', lr),
        ('svm', svm),
        ('knn', knn)
    ],
    voting='soft'
)

# 2. Ensemble com os melhores modelos (baseado na validação cruzada)
# Evita conflito de nomes: não usar 'voting' como apelido
# (pois 'voting' é parâmetro do VotingClassifier)

top_ensemble = VotingClassifier(
    estimators=[
        ('rf',       models['rf']),
        ('gb',       models['gb']),
        ('et',       models['et']),
        ('voting_c', models['voting']),  # apelido seguro
        ('bagging',  models['bagging'])
    ],
    voting='soft'
)

# Sanity check
assert not isinstance(all_individual_models, tuple)

### Ensembles híbridos e ponderados


In [ ]:
# 3. Ensemble híbrido (combina todos + bagging)
hybrid_ensemble = VotingClassifier(
    estimators=[
        ('all_individuals', all_individual_models),
        ('top_models', top_ensemble),
        ('bagging', bagging_clf)
    ],
    voting='soft'
)

# 4. Ensemble com pesos baseados na performance
weighted_ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('et', et),
        ('voting_c', voting_clf),
        ('bagging', bagging_clf)
    ],
    voting='soft'
)

# Dicionário com todos os ensembles
ensemble_models = {
    'all_individuals': all_individual_models,
    'top_models': top_ensemble,
    'hybrid': hybrid_ensemble,
    'weighted': weighted_ensemble
}


## Avaliação de ensembles e treinamento final


In [ ]:
# Teste e seleção do melhor ensemble
print("Testando diferentes sistemas de votação:")
for name, ensemble in ensemble_models.items():
    if isinstance(ensemble, tuple):  # fallback defensivo
        ensemble = ensemble[0]
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', ensemble)
    ])
    scores = cross_val_score(pipeline, X_engineered, y, cv=cv, scoring='accuracy', error_score='raise')
    mean_score = scores.mean()
    std_score = scores.std()
    print(f"{name}: {mean_score:.4f} (+/- {std_score:.4f})")
    if mean_score > best_score:
        best_score = mean_score
        best_model = name
        best_pipeline = pipeline
print(f"\nMelhor sistema de votação: {best_model} com score: {best_score:.4f}")
# Treina o melhor ensemble (ou melhor pipeline anterior)
if 'best_pipeline' in locals():
    best_pipeline.fit(X_engineered, y)
else:
    print("Nenhum ensemble superou o melhor modelo individual. Usando pipeline anterior.")
    best_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', models[best_model])
    ])
    best_pipeline.fit(X_engineered, y)

Testando diferentes sistemas de votação:
all_individuals: 0.7708 (+/- 0.0276)


ValueError: Estimator names conflict with constructor arguments: ['voting']

## Predição em teste e submissão


In [ ]:
# Predições para o conjunto de teste
pred_test = best_pipeline.predict(test_engineered)

print(f"\n   PREDIÇÕES   ")
print(f"Distribuição das predições: {np.bincount(pred_test)}")

# Gera arquivo de submissão
submission = pd.read_csv('data/sample_submission.csv')
submission['labels'] = pred_test
submission.to_csv('submission.csv', index=False)

print("Arquivo de submissão salvo como 'submission.csv'")